In [63]:
import numpy as np
import pandas as pd
import talib
import numpy
import datetime as dt
import backtrader as bt

In [2]:
df= pd.read_excel(r'gbp_usd_1h.xlsx')

In [3]:
df['atr'] = talib.ATR(
    df['high'],
    df['low'],
    df['close'],
    timeperiod=14
)

In [4]:
df['ma'] = talib.KAMA(
    df['close'],
    timeperiod=14
)

In [5]:
df = df.iloc[25:]

In [6]:
df : pd.DataFrame= df.reset_index(drop = True)

In [7]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [8]:
df['timestamp'] = df['datetime'].apply(lambda d: int(d.timestamp()))

In [9]:
array = numpy.array(df[['timestamp','close','low', 'atr']])

In [10]:
array[:,3]

array([0.00223147, 0.00214137, 0.00211484, ..., 0.0015302 , 0.00165161,
       0.00172364], shape=(35566,))

In [11]:
patterns = []
array_size = array.shape[0]
for i,element in enumerate(array):
    if i == array_size - 7:
        break

    pat = array[i:i+7,:].copy()

    pat[:,1] = pat[:,1] - min(pat[:,2])
    pat[:,1] = pat[:,1] / pat[:,3]
    pat = pat[:, 1]
    patterns.append(pat)

In [12]:
any_pat = patterns[10]

In [13]:
X = any_pat[0:6]

In [14]:
Y = any_pat[6]

In [15]:
patterns = numpy.array(patterns)

In [16]:
final_df = pd.DataFrame(patterns)

In [22]:
final_df = final_df.sample(frac = 1).reset_index(drop = True)

In [35]:
from sklearn.model_selection import train_test_split

X = patterns[:,0:6]
y = patterns[:,6]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
from lightgbm import LGBMRegressor


In [42]:
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.05,
    num_leaves=31,
    subsample=0.8,
    colsample_bytree=0.8
)

In [43]:
scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=kf)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000482 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 28447, number of used features: 6
[LightGBM] [Info] Start training from score 1.369288


C:\Users\m.rahimi\PycharmProjects\data_labs\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000416 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 28447, number of used features: 6
[LightGBM] [Info] Start training from score 1.372398


C:\Users\m.rahimi\PycharmProjects\data_labs\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000351 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 28447, number of used features: 6
[LightGBM] [Info] Start training from score 1.366052


C:\Users\m.rahimi\PycharmProjects\data_labs\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000441 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 28447, number of used features: 6
[LightGBM] [Info] Start training from score 1.370788


C:\Users\m.rahimi\PycharmProjects\data_labs\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000397 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1530
[LightGBM] [Info] Number of data points in the train set: 28448, number of used features: 6
[LightGBM] [Info] Start training from score 1.370199


C:\Users\m.rahimi\PycharmProjects\data_labs\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [44]:
scores

array([-0.46391484, -0.46060551, -0.46111725, -0.46118553, -0.4650291 ])

In [48]:
max(patterns[:,0])

np.float64(19.560267196618554)

In [62]:
patterns[patterns[:, 0] > 1, :]

array([[1.45637453, 1.87926362, 1.03451751, ..., 0.96864778, 0.5028171 ,
        0.27840862],
       [2.3826378 , 1.53051905, 1.06963891, ..., 1.03077505, 0.81975871,
        0.9259281 ],
       [2.60282715, 2.14401074, 2.59284507, ..., 1.99010606, 2.10014976,
        0.39326461],
       ...,
       [1.22216103, 1.62713006, 1.58742383, ..., 1.00873749, 0.90960168,
        1.27345095],
       [2.44069508, 2.41108714, 2.37412332, ..., 1.7230666 , 2.07850614,
        1.9082511 ],
       [2.41108714, 2.37412332, 1.84935206, ..., 2.07850614, 1.9082511 ,
        2.91836194]], shape=(18843, 7))

In [67]:
class CustomPandasData(bt.feeds.PandasData):
    lines = ('atr','adx', 'ma', 'id')  # add new lines
    params = (
        ('atr', -1),
        ('adx', -1),
        ('ma', -1),
        ('id', -1),
    )


In [78]:
class StrategyB(bt.Strategy):

    def __init__(self):
        self.trades = []
        self.orders = []
        self._open_trades = {}

        self.call_counter = 0

    def calculate_volume(
            self,
            entry_price: float,
            stop_price: float,
            balance: int = 10000,
            risk: int = 1,
    ):
        print("call_size")
        self.call_counter += 1
        risk_amount = balance * (risk / 100)
        per_unit_risk = abs(entry_price - stop_price)
        units = risk_amount / per_unit_risk
        return int(units)

    def next(self):
        cal_ma_status = lambda close, ma: 'upper' if close > ma else 'under'

        ma = cal_ma_status(self.data.close[0], self.data.ma[0])

        if ma == "upper":
            price = self.data.close[0]
            entry_price = price
            stop_loss = entry_price - self.data.atr[0] * 2.4
            take_profit = entry_price + self.data.atr[0] * 2.4
            size = self.calculate_volume(
                    entry_price,
                    stop_loss,
                    risk=2
                )
            self.buy_bracket(
                price=entry_price,
                stopprice=stop_loss,
                limitprice=take_profit,
                size=size,
                tradeid = self.data.id[0]
            )

        # if current_ma_status == "under":
        #     price = self.data.close[0]
        #     entry_price = price
        #     stop_loss = entry_price + self.data.atr[0] * 2.4
        #     take_profit = entry_price - self.data.atr[0] * 4.8
        #     size = self.calculate_volume(
        #             entry_price,
        #             stop_loss,
        #             risk=2
        #         )
        #     self.sell_bracket(
        #         price=entry_price,
        #         stopprice=stop_loss,
        #         limitprice=take_profit,
        #         size=size
        #     )

    def notify_trade(self, trade : bt.trade.Trade):
        self.trades.append(trade)


    def notify_order(self, order):
        self.orders.append(order)



In [79]:
data = CustomPandasData(dataname=df.set_index("datetime"))

cerebro = bt.Cerebro()
cerebro.adddata(data)
cerebro.addstrategy(StrategyB)
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade')
cerebro.broker.setcash(10000)
cerebro.broker.set_coo(True)
cerebro.broker.setcommission(commission=0.0, leverage=500)
result = cerebro.run()

call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size
call_size


In [80]:
cerebro.broker.getvalue()

10950.818744543096